In [1]:
import sqlite3
import pandas as pd
import polars as pl
import os
import numpy as np

DB_PATH = CARPETA_DATA = r"./Data/" + "esports_data.db"
OUTPUT_PRO_FILE = "draft_oracle_pro_signatures.parquet"

OUTPUT_PRO_FILE = "draft_oracle_pro_signatures.parquet"

def generate_pro_signatures():
    print("🏆 Extrayendo Firmas de Jugadores Profesionales...")

    if not os.path.exists(DB_PATH):
        raise FileNotFoundError(f"No encuentro {DB_PATH}")

    conn = sqlite3.connect(DB_PATH)

    # 1. QUERY: Unimos Partidas + Jugadores para saber quién jugó qué
    query = """
    SELECT
        p.nickname as player_name,
        p.role as position,
        m.champion_name,
        m.win,
        m.kills,
        m.deaths,
        m.assists,
        m.gold,
        m.damage_dealt
    FROM match_history m
    JOIN players p ON m.player_id = p.id
    WHERE m.champion_name IS NOT NULL
    """

    # Cargar en Polars
    df_raw = pl.from_pandas(pd.read_sql(query, conn))
    conn.close()

    print(f"   -> Partidas Procesadas: {df_raw.height}")

    # 2. CALCULAR "MAESTRÍA" (PROFICIENCY)
    # Agrupamos por Jugador y Campeón
    pro_stats = df_raw.group_by(["player_name", "champion_name"]).agg([
        pl.count("win").alias("games_played"),
        pl.col("win").mean().alias("pro_winrate"),

        # KDA = (K + A) / Max(1, D)
        ((pl.col("kills").mean() + pl.col("assists").mean()) /
         (pl.col("deaths").mean().fill_null(1).clip(1, 99))).alias("pro_kda"),

        # Oro promedio (Eficiencia)
        pl.col("gold").mean().alias("avg_gold")
    ])

    # 3. SCORE DE CONFIANZA (LA FÓRMULA MAGICA)
    # Un 100% de WR en 1 partida no vale nada. Un 60% en 50 partidas vale oro.
    # Fórmula: Winrate * Factor de Experiencia Logarítmico

    pro_stats = pro_stats.with_columns(
        (pl.col("pro_winrate") * (1 - (1 / (pl.col("games_played") + 1).log().clip(1, 100)))).alias("proficiency_score")
    )

    # Normalización del nombre del campeón para búsquedas (todo minuscula)
    pro_stats = pro_stats.with_columns(
        pl.col("champion_name").str.to_lowercase().alias("champ_key")
    )

    print(f"   💾 Firmas Guardadas: {pro_stats.shape} registros.")
    pro_stats.write_parquet(OUTPUT_PRO_FILE)
    return pro_stats

# EJECUTAR
df_pro = generate_pro_signatures()
print(df_pro.head(5))

🏆 Extrayendo Firmas de Jugadores Profesionales...
   -> Partidas Procesadas: 32197
   💾 Firmas Guardadas: (6288, 8) registros.
shape: (5, 8)
┌────────────┬────────────┬────────────┬────────────┬──────────┬──────────┬────────────┬───────────┐
│ player_nam ┆ champion_n ┆ games_play ┆ pro_winrat ┆ pro_kda  ┆ avg_gold ┆ proficienc ┆ champ_key │
│ e          ┆ ame        ┆ ed         ┆ e          ┆ ---      ┆ ---      ┆ y_score    ┆ ---       │
│ ---        ┆ ---        ┆ ---        ┆ ---        ┆ f64      ┆ f64      ┆ ---        ┆ str       │
│ str        ┆ str        ┆ u32        ┆ f64        ┆          ┆          ┆ f64        ┆           │
╞════════════╪════════════╪════════════╪════════════╪══════════╪══════════╪════════════╪═══════════╡
│ Saint      ┆ Galio      ┆ 1          ┆ 0.0        ┆ 0.8      ┆ 9059.0   ┆ 0.0        ┆ galio     │
│ Chovy      ┆ Garen      ┆ 1          ┆ 1.0        ┆ 3.333333 ┆ 13488.0  ┆ 0.0        ┆ garen     │
│ Josedeodo  ┆ Xin Zhao   ┆ 6          ┆ 0.666667  